In [207]:
import json
import pandas as pd
import numpy as np
from ete3 import Tree, faces, AttrFace, TreeStyle, NodeStyle

In [210]:
df = pd.read_csv('../dataframes/h3n2_ha_12y_fra_egg.csv')

In [254]:
t = Tree("( ((( (((c1, c2), c1), (c3,c4), c3), ((c5,c6),c6)), c7), (c8,c9), c7), c10);")

In [255]:
clade_counts_df = df.groupby('kk_clade').size().reset_index().rename(columns={0:'clade_size'})
clade_counts = clade_counts_df.set_index('kk_clade').T.to_dict('list')

style1 = NodeStyle()
style1["fgcolor"] = "#0f0f0f"
style1["size"] = 10
style1["vt_line_color"] = "#ff0000"
style1["hz_line_color"] = "#ff0000"
style1["vt_line_width"] = 2
style1["hz_line_width"] = 2
style1["vt_line_type"] = 2 # 0 solid, 1 dashed, 2 dotted
style1["hz_line_type"] = 2
for l in t.iter_leaves():
    if clade_counts[l.name] == 'c8':
        l.img_style = style1
t.render("%%inline")  

KeyError: 'c2'